# Data Science 180



![Alt text](http://thegeektown.com/wp-content/uploads/2015/03/data-scientist.jpg)




An adventure through 15 dimensions of data wrangling, visualization and modeling at mind-bending speeds.


## In a world with too much data sitting around and not enough insight, to whom will we turn for help?

![Alt](http://nextviewventures.com/blog/wp-content/uploads/2014/07/control-content-marekting-for-startups.jpg)

##YOU!    (Neo was busy...)


### You must learn how to wrangle data in the next few hours in order to save the education system. If you fail, we're all doomed...

### You have been given a dataset of test results and metadata, along with a laptop computer. 









#Good luck, everything depends on you.

 

###Where do we begin? 


###You know Python right? It does data stuff, right? OK, let's get started then.

###First thing to figure out is how to get the data files on your machine into Python in the first place.


###You've heard of this library called Pandas from another Agent -- it once saved them in a pinch. Lacking any better ideas, let's open up an editor and see if we can't at least cross the starting line.

#Reading data from disk

In [11]:
import pandas as pd
exam_data = pd.read_csv('data/PISA2009_Scored_Tests_MEX.csv')
bio_data = pd.read_csv('data/PISA2009_Questionnaire_MEX.csv')

###What did that just do? 



We called "read_csv", which presumably reads CSV files... and does what with them? 

##pd.read_csv does a magical thing 


It reads a CSV file into a DataFrame. 

DataFrames are mystical creatures in Data Science. 

Popularized by R, they provide a standardized MATRIX-style format for interacting with your data. Most data can fit into this row and column format: financial transactions, iPhone app user records, medical histories, etc.

(And you thought the Matrix references were just for fun)

![Alt](http://www.bigdataexaminer.com/wp-content/uploads/2014/12/screen-1.png)

##Since you were wondering

##Pandas has support for many formats

CSV, Text (tab separated, pipe separated, etc.), Excel, JSON, HTML, SQL, Stuff copied to your clipboard, HDF5...

## Slow down. What's really going on in the DataFrame?

## Two data structures: Series and DataFrame

###Series
Think of this as one column of your data - one data type.

### DataFrame
All of the columns in your data. Mixed data types. 



#Many Series can be combined and represented as a DataFrame object.

#A DataFrame can be represented as many Series objects. 




#Pandas provides tons of functions to:

###slice, dice, merge, join, group by, select, append, find, transform, sort, reverse, pivot and anything else you want to do




####... for both Series and DataFrames. 

Most functions are designed to work with either type or even combinations of the two, just like you would intuitively expect:

i.e. A concat function can contatenate arbitrary combinations of 0 to n Series and DataFrames.

#So you know a bit about DataFrames?

In [20]:
import pandas as pd
exam_data = pd.read_csv('data/PISA2009_Scored_Tests_MEX.csv')
bio_data = pd.read_csv('data/PISA2009_Questionnaire_MEX.csv')

#Fine. What's next?

#Exploratory Data Analysis. 

What the hell is in those files anyway?

Does it look like test data should? 

Is it completely empty? Full? Lots of missing values and NaN?

What are in the rows? columns?

Does it have appropriate features? (characteristics common to records belonging to a dataset)

###It's impossible to make good decisions moving forward until we know more

We can just output the entire dataframe to the console, but that doesn't scale beyond a couple hundred rows.

#Pandas gives us a number of tools: 


    
    .head(n)
    .info()
    .describe()

In [ ]:
exam_data.head(5)

In [ ]:
exam_data.info()

In [ ]:
exam_data.describe()

#We have two files, and both of them have a feature named 'Student ID 5-digit'


#Using this unique ID as our guide, we can match the exam scores and biographical data for a single student.

#This task comes up a lot in data wrangling, since different kinds of data will be stored in different databases. Often one of the first steps is to combine the relevant parts of each part of the data.

In [14]:
useless = {
    u' Version of cognitive database and date of release',
    u'3-character country code ',
    u'Adjudicated sub-region',
}

not_questions = {u'Booklet', 
                 u'School ID 5-digit', 
                 u'Student ID 5-digit',
                 u'OECD country',
                 u'Country code ISO 3-digit',}

score_mapping = {
    'Score 0': 0,
    'Score 1': 1,
    'Score 2': 2,
    'Not reached': 0,
}

questions = set(exam_data.columns) - not_questions - useless

for question in questions:
    exam_data[question] = exam_data[question].map(score_mapping)

In [15]:
math_qs = {q for q in questions if q.startswith('MATH')}
read_qs = {q for q in questions if q.startswith('READ')}
scie_qs = {q for q in questions if q.startswith('SCIE')}
    
totals = exam_data[list(questions)].sum(axis=1)
math_score = exam_data[list(math_qs)].sum(axis=1)
read_score = exam_data[list(read_qs)].sum(axis=1)
scie_score = exam_data[list(scie_qs)].sum(axis=1)

#Let's do a merge to get our data into a single managable file

In [16]:
score_df = pd.DataFrame({'Total Score': totals, 
                         'Math Score': math_score, 
                         'Reading Score': read_score, 
                         'Science Score': scie_score,
                         'Student ID 5-digit': exam_data['Student ID 5-digit']})

In [17]:
data = pd.merge(score_df, bio_data, on='Student ID 5-digit')


#Now we can pick any feature (read: column) and get information on it:

In [18]:
data['Reading Enjoyment Time'].value_counts()

30 minutes or less a day      16997
I don�t read for enjoyment     9071
Between 30 and 60 minutes      7047
1 to 2 hours a day             3772
More than 2 hours a day        1008
dtype: int64

#But what about relationships between variables?

We could look at sets of rows and see what occurs together, or compute statistics of co-occurrence ---> but still impossible to get a comprehensive view quickly.

What can we try?

#Let's do some plots!

#Matplotlib

- The historical go-to for plotting
- allows lots of fine-grained control
- built with numpy in mind

#Seaborn

- Expressive power
- built with pandas in mind

We will mainly use seaborn examples in this presentation

In [19]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


###Jupyter notebooks can display graphics inline

Blah blah

In [ ]:
import seaborn as sns

#Scatterplot

View relationship between two continuous variables

In [ ]:
sns.jointplot(data['Index of economic, social and cultural status (WLE)'], 
              data['Home Possessions'], kind="hex")

In this dataset, several variables can stand as proxies for socio-economic status

# Histogram


- Visualize distribution of continuous data.


- Visualize distribution across categorical levels.

- Can plot two histograms on top of each other

- See the effects of the variable on the target

In [ ]:
groups = data.groupby('Sex').groups
for key, row_ids in groups.iteritems():
    pylab.hist(data['Total Score'][row_ids].values,
               normed=True,
               bins=np.linspace(0, 70, 11),
               alpha=0.35,
               label=str(key))
pylab.legend()

###Doesn't work as well for more than two levels

In [ ]:
groups = data.groupby('Grade').groups
for key, row_ids in groups.iteritems():
    pylab.hist(data['Total Score'][row_ids].values,
               normed=True,
               bins=np.linspace(0, 70, 11),
               alpha=0.35,
               label=str(key))
pylab.legend()

## Violin Plots work better for comparing several distributions

In [ ]:
nonnull_subset = data['Total Score'].notnull()
plt.figure(figsize=(12, 6))
sns.violinplot(data['Total Score'][nonnull_subset], 
               data['Father  <Highest Schooling>'][nonnull_subset], 
               inner='box',
               bw=1)

##Alternatively, use `FacetGrid`
###Visualize more effect of two variables

In [ ]:
g = sns.FacetGrid(data, row="Sex", col="At Home - Mother", margin_titles=True)
bins = np.linspace(0, 67, 13)
g.map(plt.hist, "Total Score", color="steelblue", bins=bins, lw=0, normed=True)

###If distribution not required, try factor plot

In [ ]:
g = sns.factorplot("At Home - Father", "Total Score", "Sex",
                    data=data, kind="bar",
                    size=6, palette="muted", dropna=True)
g.despine(left=True)
g.set_ylabels("Mean Score")

In [ ]:
g = sns.factorplot("Sex", "Total Score", "Sex",
                   row="At Home - Mother",
                   col="At Home - Father",
                   data=data, kind="bar",
                   size=6, palette="muted",
                   dropna=True)
g.despine(left=True)
g.set_ylabels("Mean Score")

#Heatmaps

In [ ]:
ptable = pd.pivot_table(
    data, 
    values='Total Score', 
    index='Like Read - Fiction', 
    columns='Like Read - Non-fiction books')
sns.heatmap(ptable, annot=True, fmt="f")


###Not very useful if not in order...

###Heatmaps - Round 2
####Effects of variables over a range

In [ ]:
display_order = [
    u'Never or almost never',
    u'A few times a year', 
    u'About once a month',                   
    u'Several times a month', 
    u'Several times a week'
]
display_table = ptable[display_order].reindex(reversed(display_order))
sns.heatmap(display_table,
            annot=True, 
            fmt="f")

###Pivot tables can do other aggregations

In [ ]:
count_table = pd.pivot_table(
    data, 
    values='Total Score', 
    index='Like Read - Fiction', 
    columns='Like Read - Non-fiction books',
    aggfunc=np.count_nonzero)

sns.heatmap(count_table[display_order].reindex(reversed(display_order)), annot=True, fmt="f")

#Scatterplot

In [ ]:
import random
some_sample = random.sample(data.index, 1000)
sns.lmplot("Total Score", 
           'Index of economic, social and cultural status (WLE)', 
           data.ix[some_sample])

Why plot just a subset of the data?

In [ ]:
sns.lmplot("Total Score", 
           'Index of economic, social and cultural status (WLE)', 
           data)

In [ ]:
bad_data = data['Reading Enjoyment Time'].apply(lambda x: isinstance(x, basestring) and x.startswith('I don'))
reading_enjoyment = data['Reading Enjoyment Time'].copy()
reading_enjoyment[bad_data] = 'No Joy'

time_x_fiction = pd.pivot_table(
    data, 
    values='Total Score', 
    index='Like Read - Fiction', 
    columns=reading_enjoyment)

col_order = [
    'No Joy', 
    '30 minutes or less a day', 
    'Between 30 and 60 minutes',
    '1 to 2 hours a day',
    'More than 2 hours a day'
]

row_order = [
    'Several times a week',
    'Several times a month',
    'About once a month',
    'A few times a year',
    'Never or almost never',
]

display_table = time_x_fiction[col_order].reindex(row_order)
sns.heatmap(display_table, annot=True)


In [ ]:
bad_data = data['Reading Enjoyment Time'].apply(lambda x: isinstance(x, basestring) and x.startswith('I don'))
reading_enjoyment = data['Reading Enjoyment Time'].copy()
reading_enjoyment[bad_data] = 'No Joy'

time_x_nonfiction = pd.pivot_table(
    data, 
    values='Total Score', 
    index='Like Read - Non-fiction books', 
    columns=reading_enjoyment)

col_order = [
    'No Joy', 
    '30 minutes or less a day', 
    'Between 30 and 60 minutes',
    '1 to 2 hours a day',
    'More than 2 hours a day'
]

row_order = [
    'Several times a week',
    'Several times a month',
    'About once a month',
    'A few times a year',
    'Never or almost never',
]

non_f_display_table = time_x_nonfiction[col_order].reindex(row_order)
sns.heatmap(non_f_display_table, annot=True)

In [ ]:
key_reading = 'Online Reading'
key_score = 'Total Score'
key_econ = 'Index of economic, social and cultural status (WLE)'

data_reading = data[key_reading]
data_score = data[key_score]
data_econ = data[key_econ]

reading_bins = np.linspace(np.min(data_reading), np.max(data_reading), 5)
econ_bins = np.linspace(np.min(data_econ), np.max(data_econ), 5)

to_pivot = pd.DataFrame({
    key_reading: np.digitize(data_reading, 
                             bins=reading_bins),
    key_econ: np.digitize(data_econ,
                          bins=econ_bins),
    key_score: data_score
})

ptable = pd.pivot_table(
    to_pivot, 
    values=key_score,
    index=key_reading,
    columns=key_econ,
    aggfunc=np.mean)
ptable.columns = pd.Series(map(str, econ_bins), name='Economic Status')
ptable.index = pd.Series(map(str, reading_bins), name='Reading Values')

sns.heatmap(ptable, annot=False, fmt="f")

#Modeling

In [ ]:
import sklearn
from sklearn import linear_model

# To look at the features again:
#list(data.columns.values)
print '****************'
print "What about missing values?"
print "We can insert the mean!"
#data.mean()
print '****************'
good_rows = data[['Home Possessions','Grade', 'Grade compared to modal grade in country', 'Math Score', 'Reading Score', 'Total Score']].fillna(data.mean())

#good_rows.describe()
#good_rows.info()

new_data = good_rows[['Home Possessions','Grade', 'Grade compared to modal grade in country', 'Math Score', 'Reading Score']]

training_target = good_rows['Total Score'][:-5000]
training = new_data[:-5000]

validation_target = good_rows['Total Score'][-5000:]
validation = new_data[-5000:]

lr = linear_model.LinearRegression()

lr.fit(training, training_target)

# #pd.unique(good_rows.values.ravel())
# print "NUll: {0} \n".format(good_rows.isnull().sum())
# print "***************"
# predicted = sklearn.cross_validation.cross_val_predict(linear_model.LinearRegression(), new_data, target, cv=2)

# fig,ax = matplotlib.pyplot.subplots()
# ax.scatter(target, predicted)
# fig.show()

# The coefficients
print('Coefficients: \n', lr.coef_)
# The root mean square error
print("RMSE: %.2f"
      % np.sqrt(np.mean(lr.predict(validation) - validation_target) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lr.score(validation, validation_target))

# Plot outputs
print validation.shape, validation_target.shape

lr_output = pd.concat([pd.DataFrame(lr.predict(validation), columns=['Predicted']), validation_target], axis=1)
lr_output.info()

matplotlib.pyplot.scatter(lr.predict(validation), validation_target,  color='red')
matplotlib.pyplot.plot([0,60], [0,60], color='blue',
         linewidth=3)

#matplotlib.pyplot.xticks(())
#matplotlib.pyplot.yticks(())

matplotlib.pyplot.show()

#sns.jointplot('Predicted', 'Total Score', data=lr_output, kind="reg",
#                  xlim=(0, 60), ylim=(0, 20), color=color, size=7)

In [ ]:
#sklearn.metrics.log_loss(target, predicted)
print "look at log_loss?"

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(training, training_target)

# fig1,ax1 = matplotlib.pyplot.subplots()
# ax1.scatter(target,clf.predict(new_data))
# fig1.show()
#target
clf_predict = clf.predict(validation)


In [ ]:
#clf_predicted_actual = pd.concat([clf_predict, target], axis=1, keys=['predicted', 'actual'])

In [ ]:
color = sns.color_palette()[2]
#g = sns.jointplot('predicted', 'actual', data=clf_predicted_actual, kind="reg",
#                  xlim=(0, 60), ylim=(0, 20), color=color, size=7)

In [ ]:
# Create default Random Forest Regression Model
from sklearn.ensemble.forest import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(training, training_target)


rf_predictions = pd.DataFrame(rf.predict(validation), columns=['Predicted Math Score'])
rf_validation_target = pd.DataFrame(validation_target, columns=['Total Score']).reset_index()
rf_predicted_actual = pd.concat([rf_predictions, rf_validation_target], axis=1)
print rf_predicted_actual

In [ ]:
pd.__version__

In [ ]:
color = sns.color_palette()[1]
g = sns.jointplot('Predicted Math Score', 'Total Score', data=rf_predicted_actual, kind="reg",
                  xlim=(0, 25), ylim=(0, 25), color=color, size=10)

In [ ]:
score = rf.score(validation, validation_target)
print "Score: {}".format(score)
print rf.feature_importances_, new_data.info()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

colors = ['','red','blue','green','red','blue','green']
for degree in [1, 2, 6]:
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(training, training_target)
    y_plot = model.predict(validation)
    plt.scatter(validation_target, y_plot, color=colors[degree],label="degree %d" % degree)
    matplotlib.pyplot.plot([0,60], [0,60], color='black',linewidth=3)
    matplotlib.pyplot.legend(loc='lower left')
    matplotlib.pyplot.show()
    # The mean square error
    print("RMSE: %.2f" % np.sqrt(np.mean((y_plot - validation_target) ** 2)))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % model.score(validation, validation_target))

In [ ]:
from sklearn.svm import SVR
import matplotlib.pyplot as plt

# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(training, training_target).predict(validation)
y_lin = svr_lin.fit(training, training_target).predict(validation)
y_poly = svr_poly.fit(training, training_target).predict(validation)

plt.scatter(training, training_target, c='k', label='data')
plt.hold('on')
plt.plot(training, y_rbf, c='g', label='RBF model')
plt.plot(training, y_lin, c='r', label='Linear model')
plt.plot(training, y_poly, c='b', label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()



#Classifcation

"How do we do that?"
"Our target becomes a discrete variable"
"Our training data can stay the same"